In [1]:
import random
import shutil
import argparse
import sklearn
from pathlib import Path
import numpy as np
import os

In [2]:
def load_image( status=None):
    path = "/Users/srijan/Desktop/lungnodule/"
    x = np.squeeze(np.load(os.path.join(path, 'x_'+status+'_64x64x32.npy')))
    y = np.squeeze(np.load(os.path.join(path, 'm_'+status+'_64x64x32.npy')))
    x = np.expand_dims(x, axis=1)
    y = np.expand_dims(y, axis=1)

    return x, y

In [3]:
x_train, y_train = load_image( 'train')
print('x_train: {} | {} ~ {}'.format(x_train.shape, np.min(x_train), np.max(x_train)))
print('y_train: {} | {} ~ {}'.format(y_train.shape, np.min(y_train), np.max(y_train)))

x_valid, y_valid = load_image( 'valid')
print('x_valid: {} | {} ~ {}'.format(x_valid.shape, np.min(x_valid), np.max(x_valid)))
print('y_valid: {} | {} ~ {}'.format(y_valid.shape, np.min(y_valid), np.max(y_valid)))

x_test, y_test = load_image( 'test')
print('x_test: {} | {} ~ {}'.format(x_test.shape, np.min(x_test), np.max(x_test)))
print('y_test: {} | {} ~ {}'.format(y_test.shape, np.min(y_test), np.max(y_test)))

x_train: (4082, 1, 64, 64, 32) | 0.0 ~ 1.0
y_train: (4082, 1, 64, 64, 32) | 0 ~ 1
x_valid: (3126, 1, 64, 64, 32) | 0.0 ~ 1.0
y_valid: (3126, 1, 64, 64, 32) | 0 ~ 1
x_test: (852, 1, 64, 64, 32) | 0.0 ~ 1.0
y_test: (852, 1, 64, 64, 32) | 0 ~ 1


In [4]:
import import_ipynb

In [5]:
import unet_model_3d

importing Jupyter notebook from unet_model_3d.ipynb


Using TensorFlow backend.
/Users/srijan/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/srijan/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/srijan/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [6]:
import utils

importing Jupyter notebook from utils.ipynb


In [7]:
import tensorflow as tf

In [8]:
print(tf.__version__)

1.14.0


In [9]:
from keras import backend as K

In [10]:
tf.to_int=lambda x: tf.cast(x, tf.int32)
def dice_coef_loss(y_true, y_pred):
    return 1. - dice_coef(y_true, y_pred)
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = (y_pred > t)
        y_pred_ = tf.cast(y_pred_, tf.int32)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


In [15]:
model = unet_model_3d.unet_model_3d((1 , 64,64,32,) , batch_normalization = True)

In [16]:
model.compile(optimizer='adam', 
                  loss=dice_coef_loss, 
                  metrics=[mean_iou, 
                           dice_coef],
                 )

In [ ]:
model.fit(x_train, y_train,
                  validation_data=(x_valid, y_valid),
                  batch_size= 8,
                  epochs= 10, 
                  verbose=1, 
                  shuffle=True)


Train on 4082 samples, validate on 3126 samples
Epoch 1/10
1448/4082 [=========>....................] - ETA: 41:42:24 - loss: 0.6888 - mean_iou: 0.4573 - dice_coef: 0.3112